In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler ,RobustScaler
from sklearn.metrics import mean_squared_error,r2_score

# Import the data

In [42]:
SPY = pd.read_pickle('combined_cleaned_add_with_QQQ.pkl')
SPY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2055 entries, 0 to 2054
Data columns (total 46 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ds                   2055 non-null   datetime64[ns]
 1   SPY_Close            2055 non-null   float64       
 2   SPY_Volume           2055 non-null   int64         
 3   AAPL                 2055 non-null   float64       
 4   MSFT                 2055 non-null   float64       
 5   GOOG                 2055 non-null   float64       
 6   GLD                  2055 non-null   float64       
 7   SLV                  2055 non-null   float64       
 8   ^TNX                 2055 non-null   float64       
 9   DX-Y.NYB             2055 non-null   float64       
 10  JPY=X                2055 non-null   float64       
 11  EUR=X                2055 non-null   float64       
 12  USO                  2055 non-null   float64       
 13  UNG                  2055 non-nul

In [3]:
print(SPY.columns.to_list())

['ds', 'SPY_Close', 'SPY_Volume', 'AAPL', 'MSFT', 'GOOG', 'GLD', 'SLV', '^TNX', 'DX-Y.NYB', 'JPY=X', 'EUR=X', 'USO', 'UNG', 'BTC-USD', 'CPER', '^VIX', '^GDAXI', '^FTSE', '^RUT', '^N225', 'IEI', 'CNYUSD=X', '2Y_Yield', 'yield_curve', 'market_closed_count', 'yield_curve_term', 'high-low', 'before_high-low', 'SPY_RSI', 'RSI_rank', 'RSI_rank_2', 'EMA_20', 'EMA_50', 'EMA_200', 'EMA_20_50', 'EMA_50_200', 'EMA_50_diff', 'EMA_200_diff', 'SPY_std', 'SPY_mean', 'SPY_30', 'QQQ_Close', 'QQQ_Volume', 'qqq_std', 'qqq_mean']


In [43]:
def calculate_rsi(prices, period=14):
    prices = pd.to_numeric(prices, errors='coerce')
    delta  = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

  # MACD Calculation
def calculate_macd(prices, fast=12, slow=26):
    prices = pd.Series(prices).astype(float).dropna()
    exp1 = prices.ewm(span=fast, adjust=False).mean()
    exp2 = prices.ewm(span=slow, adjust=False).mean()
    return exp1 - exp2
    
def calculate_consecutive_streak(close_series: pd.Series) -> pd.Series:
    """
    Calculate the consecutive up/down streaks for a close price series.
    - If today's close > yesterday's, streak = previous_streak + 1 (or 1 if previous_streak <= 0)
    - If today's close < yesterday's, streak = previous_streak - 1 (or -1 if previous_streak >= 0)
    - If equal, streak = 0
    """
    streak = [0] * len(close_series)
    for i in range(1, len(close_series)):
        if close_series.iat[i] > close_series.iat[i - 1]:
            streak[i] = streak[i - 1] + 1 if streak[i - 1] > 0 else 1
        elif close_series.iat[i] < close_series.iat[i - 1]:
            streak[i] = streak[i - 1] - 1 if streak[i - 1] < 0 else -1
        else:
            streak[i] = 0
    return pd.Series(streak, index=close_series.index, name='consecutive_streak')


def model_own_features(
    data: pd.DataFrame,
    column: str,
    keep_columns = None
) -> pd.DataFrame:
    """
    Add RSI, MACD, rolling means, std, and consecutive streak for `column`.
    Optionally preserve only keep_columns + new features.
    """
    df = data.copy()
    # Ensure numeric
    df[column] = pd.to_numeric(df[column], errors='coerce')
    
    # 1) RSI
    df['RSI_14'] = calculate_rsi(df[column], period=14)
    
    # 2) MACD
    #macd_df = calculate_macd(df[column])
    #df = df.join(macd_df)
    df['MACD'] = calculate_macd(df[column])
    # 3) Rolling stats
    df['30d_mean'] = df[column].rolling(30).mean()
    df['5d_mean']  = df[column].rolling(5).mean()
    df['5d_std']   = df[column].rolling(5).std()
    
    # 4) Consecutive streak
    df['streak'] = calculate_consecutive_streak(df[column])
    
    # 5) Select output columns
    if keep_columns is not None:
        # always include the new features
        new_feats = ['RSI_14', 'MACD', '30d_mean', '5d_mean', '5d_std', 'streak']
        df = df[ keep_columns + new_feats ]

    df = df.dropna()

    return df


In [44]:
result = model_own_features(
    data=SPY,
    column='SPY_Close',
    keep_columns=['SPY_Close','high-low','before_high-low']
)



result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2026 entries, 29 to 2054
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SPY_Close        2026 non-null   float64
 1   high-low         2026 non-null   float64
 2   before_high-low  2026 non-null   float64
 3   RSI_14           2026 non-null   float64
 4   MACD             2026 non-null   float64
 5   30d_mean         2026 non-null   float64
 6   5d_mean          2026 non-null   float64
 7   5d_std           2026 non-null   float64
 8   streak           2026 non-null   int64  
dtypes: float64(8), int64(1)
memory usage: 158.3 KB


In [51]:
result = result.reset_index(drop=True)

result.head()

,SPY_Close,high-low,before_high-low,RSI_14,MACD,30d_mean,5d_mean,5d_std,streak
0,2358.570068,26.150146,22.649902,47.955152,-2.163017,2364.098332,2347.710010,6.574907,1
1,2361.129883,10.420166,26.150146,48.276382,-1.700607,2364.883325,2350.245996,8.948561,2
2,2368.060059,11.839844,10.420166,47.892270,-0.766106,2365.510327,2354.666016,11.419225,3
3,2362.719971,7.750000,11.839844,45.208163,-0.451207,2366.026994,2358.414014,10.025371,-1
4,2358.840088,21.140137,7.750000,46.940722,-0.508855,2366.283000,2361.864014,3.861890,-2


### Creat a sequence

In [17]:
def create_sequences_multifeature(data, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i - window_size:i])
        y.append(data[i, 0])  # תחזית רק למחיר
    return np.array(X), np.array(y)

window_size = 30
#X, y = create_sequences_multifeature(scaled_data, window_size)

### Scaling, Creating a sequence & spliting the data

In [46]:
scaler = MinMaxScaler() # scale the data
def prepare_RNN_data(data,window_size,scaler,split_ratio=0.8):
    
    scaled_data = scaler.fit_transform(data)

    X, y = create_sequences_multifeature(scaled_data, window_size)
    split = int(split_ratio * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    return X_train, X_test, y_train, y_test

In [47]:
window_size = 30
X_train, X_test, y_train, y_test = prepare_RNN_data(result, 30,scaler) # for sequence length of 30

# Search for the best parameters

In [29]:
import itertools

def search_best_model(
    model_type: str,
    X_train, y_train,
    X_val, y_val,
    window_size: int,
    layer_options=(1, 2, 3),
    unit_options=(32, 64, 98),
    epochs=20,
    batch_size=32,
    patience=3,
    learning_rate=0.001
):
    """
    Grid‐search over small GRU/LSTM architectures to minimize validation MSE.
    
    Parameters
    ----------
    model_type : {"GRU", "LSTM"}
        Which recurrent cell to use.
    X_train, y_train : np.ndarray
        Training data.
    X_val, y_val : np.ndarray
        Validation data.
    window_size : int
        Number of timesteps in each input sequence.
    layer_options : iterable of int
        You’ll try each of these as possible numbers of recurrent layers.
    unit_options : iterable of int
        Possible unit‐counts for each layer.
    epochs : int
        Maximum epochs per architecture.
    batch_size : int
    patience : int
        EarlyStopping patience on val_loss.
    learning_rate : float

    Returns
    -------
    best_model : keras.Model
        The model instance that achieved the lowest val_loss.
    best_val_loss : float
        Its validation loss.
    best_config : (n_layers, units_tuple)
        The layer‐count and the specific units in each layer.
    """
    if model_type not in ("GRU", "LSTM"):
        raise ValueError(f"model_type must be 'GRU' or 'LSTM', got {model_type!r}")
    Cell = GRU if model_type == "GRU" else LSTM

    best_val_loss = float("inf")
    best_model     = None
    best_config    = None

    for n_layers in layer_options:
        for units_combo in itertools.product(unit_options, repeat=n_layers):
            # Build model
            model = Sequential()
            for i, units in enumerate(units_combo):
                return_seq = (i < n_layers - 1)
                if i == 0:
                    model.add(
                        Cell(
                            units,
                            return_sequences=return_seq,
                            input_shape=(window_size, X_train.shape[2])
                        )
                    )
                else:
                    model.add(Cell(units, return_sequences=return_seq))
            model.add(Dense(1))

            # Compile & train
            opt = Adam(learning_rate=learning_rate)
            model.compile(optimizer=opt, loss="mse")
            es = EarlyStopping(
                monitor="val_loss",
                patience=patience,
                restore_best_weights=True
            )
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=epochs,
                batch_size=batch_size,
                callbacks=[es],
                verbose=0
            )

            val_loss = min(history.history["val_loss"])
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_config   = (n_layers, units_combo)
                best_model    = model

    return best_model, best_val_loss, best_config


In [ ]:
best_model, best_loss, best_cfg = search_best_model(
    model_type="GRU",                # or "LSTM"
    X_train=X_train, y_train=y_train,
    X_val=X_test,  y_val=y_test,
    window_size=window_size,
    layer_options=[1,2,3],
    unit_options=[32,64,98],
    epochs=20,
    batch_size=32,
    patience=3,
    learning_rate=0.001
)
print("Best loss:", best_loss, "Config:", best_cfg)


# Training function

In [48]:


def train_lstm(
    X_train, y_train,
    learning_rate=0.001,
    epochs=20,
    num_layers=1,
    units=(64,),
    window_size=None,
    batch_size=32,
    validation_split=0.2,
    patience=3,
    checkpoint_path='best_lstm.h5',
    dense_units=None,
    dense_activation='relu'
):
    """
    Builds, compiles, and trains a multi-layer LSTM model, with optional intermediate Dense layer.

    Parameters:
    - X_train, y_train: training data arrays
    - learning_rate: float, optimizer learning rate
    - epochs: int, max training epochs
    - num_layers: int, number of LSTM layers
    - units: tuple of ints, units per LSTM layer (length must equal num_layers)
    - window_size: int, sequence length (timesteps)
    - batch_size: int
    - validation_split: float
    - patience: int, early stopping patience
    - checkpoint_path: str, filepath to save best model
    - dense_units: int or None, if specified adds Dense(dense_units, activation=dense_activation)
    - dense_activation: str, activation for the optional Dense layer

    Returns:
    - model: trained Keras model
    - history: training history
    """
    if num_layers != len(units):
        raise ValueError(f"num_layers ({num_layers}) must equal length of units tuple ({len(units)})")
    
    opt = Adam(learning_rate=learning_rate)
    model = Sequential()
    
    # Add LSTM layers
    for i, u in enumerate(units):
        return_seq = (i < num_layers - 1)
        if i == 0:
            model.add(LSTM(u, return_sequences=return_seq, input_shape=(window_size, X_train.shape[2])))
        else:
            model.add(LSTM(u, return_sequences=return_seq))
    
    # Optional Dense layer
    if dense_units is not None:
        model.add(Dense(dense_units, activation=dense_activation))
    # Output layer
    model.add(Dense(1))
    
    model.compile(optimizer=opt, loss='mse')
    
    es = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    chk = ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss',
                          save_best_only=True, verbose=1)
    
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=[es, chk],
        verbose=1
    )
    return model, history

def train_gru(
    X_train, y_train,
    learning_rate=0.001,
    epochs=20,
    num_layers=1,
    units=(64,),
    window_size=None,
    batch_size=32,
    validation_split=0.2,
    patience=3,
    checkpoint_path='best_gru.h5',
    dense_units=None,
    dense_activation='relu'
):
    """
    Builds, compiles, and trains a multi-layer GRU model (on GPU if available), with optional Dense layer.

    Parameters: same as train_lstm.
    """
    if num_layers != len(units):
        raise ValueError(f"num_layers ({num_layers}) must equal length of units tuple ({len(units)})")
    
    # Choose device
    device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
    with tf.device(device):
        opt = Adam(learning_rate=learning_rate)
        model = Sequential()
        # Add GRU layers
        for i, u in enumerate(units):
            return_seq = (i < num_layers - 1)
            if i == 0:
                model.add(GRU(u, return_sequences=return_seq, input_shape=(window_size, X_train.shape[2])))
            else:
                model.add(GRU(u, return_sequences=return_seq))
        # Optional Dense layer
        if dense_units is not None:
            model.add(Dense(dense_units, activation=dense_activation))
        # Output layer
        model.add(Dense(1))
        model.compile(optimizer=opt, loss='mse')
    
    es = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    chk = ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss',
                          save_best_only=True, verbose=1)
    
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=[es, chk],
        verbose=1
    )
    return model, history


### LSTM

In [105]:
model_lstm = train_lstm(
    X_train, y_train,
    learning_rate=0.004,
    epochs=30,
    num_layers=1,
    units=(90,),
    window_size=30,
    batch_size=16,
    validation_split=0.2,
    patience=5,
    checkpoint_path='best_lstm.h5',
    dense_units=None,
    dense_activation='relu'
)[0]

Epoch 1/30


c:\Users\OMER\PycharmProjects\Project\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


79/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0156
Epoch 1: val_loss improved from inf to 0.00044, saving model to best_lstm.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0153 - val_loss: 4.4078e-04
Epoch 2/30
78/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.6145e-04
Epoch 2: val_loss improved from 0.00044 to 0.00034, saving model to best_lstm.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.6087e-04 - val_loss: 3.4470e-04
Epoch 3/30
78/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.0350e-04
Epoch 3: val_loss improved from 0.00034 to 0.00031, saving model to best_lstm.h5


...

80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.8224e-04 - val_loss: 2.4162e-04
Epoch 13/30
77/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5327e-04
Epoch 13: val_loss did not improve from 0.00024
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.5389e-04 - val_loss: 3.0615e-04
Epoch 14/30
77/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7135e-04
Epoch 14: val_loss did not improve from 0.00024
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.7108e-04 - val_loss: 3.7081e-04
Epoch 15/30
78/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7409e-04
Epoch 15: val_loss improved from 0.00024 to 0.00023, saving model to best_lstm.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.7363e-04 - val_loss: 2.3091e-04
Epoch 16/30
78/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5894e-04
Epoch 16: val_loss improved from 0.00023 to 0.00022, saving model to best_lstm.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.5905e-04 - val_loss: 2.1803e-04
Epoch 17/30
77/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.4652e-04
Epoch 17: val_loss improved from 0.00022 to 0.00021, saving model to best_lstm.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.4588e-04 - val_loss: 2.0622e-04
Epoch 18/30
78/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.8104e-04
Epoch 18: val_loss did not improve from 0.00021
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.8061e-04 - val_loss: 3.5404e-04
Epoch 19/30
78/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.6596e-04
Epoch 19: val_loss improved from 0.00021 to 0.00020, saving model to best_lstm.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.6562e-04 - val_loss: 1.9930e-04
Epoch 20/30
78/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5597e-04
Epoch 20: val_loss did not improve from 0.00020
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.5591e-04 - val_loss: 2.0395e-04
Epoch 21/30
78/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.3390e-04
Epoch 21: val_loss did not improve from 0.00020
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.3426e-04 - val_loss: 4.3304e-04
Epoch 22/30
72/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.4181e-04
Epoch 22: val_loss did not improve from 0.00020
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.4116e-04 - val_loss: 2.8013e-04
Epoch 23/30
79/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.8567e-04
Epoch 23: val_loss did not improve from 0.00020
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.8548e-04 - val_loss: 2.0812e-04
Epoch 24/30
75/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.4060e-04
Epoch 24: val_loss did not improve from 0.00020
80/80

### GRU

In [99]:
model_gru = train_gru(
    X_train, y_train,
    learning_rate=0.004,
    epochs=30,
    num_layers=1,
    units=(96,),
    window_size=30,
    batch_size=32,
    validation_split=0.2,
    patience=5,
    checkpoint_path='best_gru.h5',
    dense_units=None,
    dense_activation='relu'
)[0]

Epoch 1/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0375
Epoch 1: val_loss improved from inf to 0.00058, saving model to best_gru.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0370 - val_loss: 5.7643e-04
Epoch 2/30
37/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.8027e-04
Epoch 2: val_loss improved from 0.00058 to 0.00045, saving model to best_gru.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.7793e-04 - val_loss: 4.4612e-04
Epoch 3/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.0267e-04
Epoch 3: val_loss improved from 0.00045 to 0.00025, saving model to best_gru.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.0224e-04 - val_loss: 2.4518e-04
Epoch 4/30
38/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.7198e-04
Epoch 4: val_loss did not improve from 0.00025
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.7023e-04 - val_loss: 2.5169e-04
Epoch 5/30
38/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.5346e-04
Epoch 5: val_loss improved from 0.00025 to 0.00023, saving model to best_gru.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.5254e-04 - val_loss: 2.3319e-04
Epoch 6/30
38/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.4273e-04
Epoch 6: val_loss improved from 0.00023 to 0.00021, saving model to best_gru.h5


...

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.3955e-04 - val_loss: 1.9110e-04
Epoch 12/30
39/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.3652e-04
Epoch 12: val_loss did not improve from 0.00019
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.3659e-04 - val_loss: 2.0178e-04
Epoch 13/30
38/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0457e-04
Epoch 13: val_loss did not improve from 0.00019
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0566e-04 - val_loss: 1.9241e-04
Epoch 14/30
38/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.3265e-04
Epoch 14: val_loss did not improve from 0.00019
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.3351e-04 - val_loss: 2.3435e-04
Epoch 15/30
34/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.4468e-04
Epoch 15: val_loss improved from 0.00019 to 0.00018, saving model to best_gru.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.4345e-04 - val_loss: 1.8489e-04
Epoch 16/30
38/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0752e-04
Epoch 16: val_loss did not improve from 0.00018
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0825e-04 - val_loss: 2.0743e-04
Epoch 17/30
39/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.2975e-04
Epoch 17: val_loss did not improve from 0.00018
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.2938e-04 - val_loss: 1.9527e-04
Epoch 18/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.1762e-04
Epoch 18: val_loss improved from 0.00018 to 0.00018, saving model to best_gru.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.1755e-04 - val_loss: 1.8089e-04
Epoch 19/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.1694e-04
Epoch 19: val_loss did not improve from 0.00018
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1690e-04 - val_loss: 1.9947e-04
Epoch 20/30
39/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.2687e-04
Epoch 20: val_loss did not improve from 0.00018
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.2723e-04 - val_loss: 1.8821e-04
Epoch 21/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.4108e-04
Epoch 21: val_loss improved from 0.00018 to 0.00018, saving model to best_gru.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.4088e-04 - val_loss: 1.7548e-04
Epoch 22/30
38/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.1848e-04
Epoch 22: val_loss did not improve from 0.00018
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1912e-04 - val_loss: 1.9594e-04
Epoch 23/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.4624e-04
Epoch 23: val_loss did not improve from 0.00018
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.4611e-04 - val_loss: 1.7950e-04
Epoch 24/30
39/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.2065e-04
Epoch 24: val_loss did not improve from 0.00018
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.2056e-04 - val_loss: 1.8043e-04
Epoch 25/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.2192e-04
Epoch 25: val_loss improved from 0.00018 to 0.00017, saving model to best_gru.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.2214e-04 - val_loss: 1.7426e-04
Epoch 26/30
37/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0345e-04
Epoch 26: val_loss did not improve from 0.00017
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0631e-04 - val_loss: 1.8468e-04
Epoch 27/30
39/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.2136e-04
Epoch 27: val_loss did not improve from 0.00017
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.2186e-04 - val_loss: 2.8769e-04
Epoch 28/30
37/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.3673e-04
Epoch 28: val_loss did not improve from 0.00017
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.3545e-04 - val_loss: 2.2967e-04
Epoch 29/30
38/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.7277e-04
Epoch 29: val_loss did not improve from 0.00017
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.7095e-04 - val_loss: 1.9321e-04
Epoch 30/30
39/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.3450e-04
Epoch 30: val_loss did not improve from 0.00017
40/40

### Load the best model

In [ ]:
from tensorflow.keras.models import load_model

# 1. Point to the same filepath you used in ModelCheckpoint
checkpoint_path = "best_lstm.h5"   # or "best_gru.h5", etc.

# 2. Load the entire model (architecture + weights + optimizer state)
best_model = load_model(checkpoint_path)

# 3. Now you can call .predict(), .evaluate(), etc.
preds = best_model.predict(X_test)

## Making a prediction and inversing the scale:

In [106]:
pred_lstm = model_lstm.predict(X_test)
pred_gru = model_gru.predict(X_test)

# reset the scaler to the original price range
price_scaler = MinMaxScaler()
price_scaler.min_, price_scaler.scale_ = scaler.min_[0], scaler.scale_[0]
pred_lstm = price_scaler.inverse_transform(pred_lstm)
pred_gru = price_scaler.inverse_transform(pred_gru)
y_test_actual = price_scaler.inverse_transform(y_test.reshape(-1, 1))
#y_test_actual = price_scaler.inverse_transform(y_test)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [107]:
mse_lstm = mean_squared_error(y_test_actual, pred_lstm)
mse_gru = mean_squared_error(y_test_actual, pred_gru)

r2_lstm = r2_score(y_test_actual, pred_lstm)
r2_gru = r2_score(y_test_actual, pred_gru)

print(f'LSTM - MSE: {mse_lstm:.4f}, R²: {r2_lstm:.4f}')
print(f'GRU  - MSE: {mse_gru:.4f}, R²: {r2_gru:.4f}')

LSTM - MSE: 8779.4122, R²: 0.9705
GRU  - MSE: 5392.4582, R²: 0.9819
